# 3 - Other Easy Wins
##### **Author: Adam Gatt**

## explicit (pre 11)

Here we have a class representing a scaling factor that you can multiply other numbers with.

In [1]:
class Factor {
    public:
    Factor(double value)
        : value(value) { }
    
    double apply(double input) const {
        return input * value;
    }
    
    private:
    double value;
};

In [2]:
double scale(double input, const Factor& factor) {
    return factor.apply(input);
};

In [3]:
scale(5, 0.5)

2.5000000

How come the call to `scale()` succeeded even though the second parameter was a double instead of a Factor? It's because of the rules C++ uses when parsing a function call. If the compiler can't find a function declaration that matches the called signature, then it will attempt a process called _implicit conversion_. This involves looking around for any other functions that it can use to convert the arguments to types that will allow the original function call to succeed. It is similar to the promotion of primitives, so that you can pass an int to a function that expects a double, except broadened to apply to other types as well. The compiler will search for any user-defined conversion functions and constructors that it can use to perform the conversion process. 

So here:
1. The call to `scale(int, double)` is initially unsuccessful as that signature has not been declared.
2. The compiler notices the declaration `scale(int, const Factor&)` and determines if it can convert the arguments to match it.
3. The compiler notices that `Factor(double value)` is available for creating a Factor out of a double.
4. The compiler inserts the implicit conversion, effectively turning the function call into `scale(5, Factor(0.5))`, and creating a temporary Factor object to submit to the scale function.

C++ performs implicit conversion not only when calling functions, [but in a range of scenarios](https://en.cppreference.com/w/cpp/language/implicit_conversion) where one type is provided where another is needed.

Initially this might seem like a benefit, for getting code to work that otherwise wouldn't. But remember that this all happens implicitly behind the scenes, without evidence in the source code itself. As a general rule, you want your application logic to be explicitly laid out to make the code more readable and verifiable.

What are the costs or risks of implicit conversion? They might include:
 * Function arguments being open to a far greater range of types than intended
 * An API that is open to guesswork rather than understanding proper usage
 * Creation of unintended temporary objects
 * Creation of objects at all that shouldn't be allowed (e.g. if the class is intended Singleton)
 * Reducing clarity of the intent of the codebase
 
How do we crack down on implicit conversion, assuming we intend to? Remember that the implicit conversion comes about due to three factors:
1. A function call with a signature that hasn't been declared
2. A candidate signature that could be matched with some conversion
3. A conversion function or constructor that can perform the conversion

Here the `explicit` keyword is provided to solve our problem by addressing factor 3. A conversion function or constructor that is marked explicit will only be used by the compiler if explicitly called in the codebase. It will not be considered during the implicit conversion process as a way of performing a needed conversion.

In [4]:
class Factor2 {
    public:
    // Now explicit, no type conversion for arguments
    explicit Factor2(int value)
        : value(value) { }
    
    int apply(int input) const {
        return input * value;
    }
    
    private:
    int value;
}

In [5]:
double scale2(double input, const Factor2& factor) {
    return factor.apply(input);
};

In [6]:
scale2(5, 0.5)

input_line_15:2:2: error: no matching function for call to 'scale2'
 scale2(5, 0.5)
 ^~~~~~
input_line_14:1:8: note: candidate function not viable: no known conversion from 'double' to 'const __cling_N56::Factor2' for 2nd argument
double scale2(double input, const Factor2& factor) {
       ^


Interpreter Error: 

Here is another use for `explicit`. In the previous notebook we had `Line myLine = NULL` (missing the `*`) and the code not only compiled, but called the constructor `Line (int thickness)`. What happened here?

The issue is that the Line class, like our original Factor class, has a constructor that the compiler considers to be a [converting constructor](https://en.cppreference.com/w/cpp/language/converting_constructor). Pre C++11 this was any constructor that accepts only a single parameter. For '11 and later, any constructor is treated as a converting constructor (due to the introduction of initializer lists).

The example below demonstrates this. The important point to know here is that despite the `=` character, the operation below is not an assignment. Instead it follows the _copy initialisation_ process, which will end up calling the constructor. _(The next section will touch briefly on copy initialisation vs direct initialisation)._

In [7]:
Factor myFactor = 0.5; // Compiles to "Factor myFactor(0.5)"
myFactor.apply(10)

5.0000000

Declaring a constructor `explicit` will prevent it from being a converting constructor, and the statement returns to attempting to assign a double to our user-defined type.

In [8]:
Factor2 myFactor2 = 0.5;
myFactor2.apply(100)

input_line_18:2:10: error: no viable conversion from 'double' to '__cling_N56::Factor2'
 Factor2 myFactor2 = 0.5;
         ^           ~~~
input_line_13:1:7: note: candidate constructor (the implicit copy constructor) not viable: no known conversion from 'double' to 'const __cling_N56::Factor2 &' for 1st argument
class Factor2 {
      ^
input_line_13:1:7: note: candidate constructor (the implicit move constructor) not viable: no known conversion from 'double' to '__cling_N56::Factor2 &&' for 1st argument


Interpreter Error: 

For extra reading, here is a 2006 (pre-11) article on this topic on [The Old New Thing](https://devblogs.microsoft.com/oldnewthing/20060524-12/?p=31083).

## Scoped enumerations

C++11 introduced "scoped" enumerations, an alternative enum syntax that can be simply achieved by adding just the one keyword "class" in the definition. Scoped enums are called so because their enum values (the "enumerants") are scoped to the enum itself rather than the class/namespace outside of it.

In [3]:
// Traditional enum declaration
enum Colour{
    RED,
    BLUE,
    GREEN
};

// New-style "scoped" enum declaration
enum class Flavour {
    VANILLA,
    CHOCOLATE,
    STRAWBERRY
};

Essentially this is a re-implementation of the enum concept with stricter rules to prevent potential errors. The new syntax is needed as previous-style enums must be left unchanged for compatibility purposes.
### Benefit: stronger type safety
Traditional "unscoped" enums map their values to some compiler-defined integral type indexed from 0, so the `GREEN` above will essentially be represented by the value 2. Traditional enums enforce no type checks when treated as (or implicitly converted to) an int. Type safety will be enforced in the other direction, however, so that the compiler won't implicitly cast an int to an enum value.

In [4]:
double calculateGST(double price) {
    return price * 1.1;
};

In [5]:
// Not even an compiler warning
calculateGST(Colour::GREEN)

2.2000000

In comparison, scoped enums are explicitly their own type and so this bug will be rejected by the compiler before it has a chance to occur.

In [6]:
calculateGST(Flavour::STRAWBERRY)

input_line_17:2:2: error: no matching function for call to 'calculateGST'
 calculateGST(Flavour::STRAWBERRY)
 ^~~~~~~~~~~~
input_line_14:1:8: note: candidate function not viable: no known conversion from 'Flavour' to 'double' for 1st argument
double calculateGST(double price) {
       ^


Interpreter Error: 

### Benefit: explicit choice of underlying data type 

If we want the enum to have an underlying type then we can specify it (as long as it is an integral type). We can then proceed to assign values to each individual enumerant as per usual.

In [7]:
enum class ChessPiece : short {
    PAWN = 1,
    KNIGHT = 3,
    ROOK = 5
};

In [8]:
short withCapture(short boardValue, ChessPiece piece) {
    return boardValue + static_cast<int>(piece);
}

withCapture(6, ChessPiece::ROOK)

11

### Benefit: tighter scope

Traditional enums are scoped to the environment surrounding the enum, so that `WRITE` below leaks its scope to `ConfigFile` and can be referenced with `ConfigFile::WRITE`. This allows the possibility of conflicts between different enums, like with the ambiguous `ConfigFile::READ`.

In [9]:
class ConfigFile {
    public:
    enum Operation{READ, WRITE};
    enum State{FOUND, OPENED, READ, FINALISED};
}

input_line_21:4:31: error: redefinition of enumerator 'READ'
    enum State{FOUND, OPENED, READ, FINALISED};
                              ^
input_line_21:3:20: note: previous definition is here
    enum Operation{READ, WRITE};
                   ^


Interpreter Error: 

Scoped enums do not leak their scope to the surrounding context. Here the two `READ` enums are `Operation::READ` and `State::READ` and cannot be confused. You can still use the class/namespace name to reference the enum value externally.  

In [10]:
class ConfigFile {
    public:
    enum class Operation{READ, WRITE};
    enum class State{FOUND, OPENED, READ, FINALISED};
};

ConfigFile::State::FOUND == ConfigFile::State::READ

false

In [11]:
// Same enum value name but strictly different values belonging to different enums.
ConfigFile::Operation::READ == ConfigFile::State::READ

input_line_24:3:29: warning: comparison of two values with different enumeration types ('__cling_N514::ConfigFile::Operation' and '__cling_N514::ConfigFile::State') [-Wenum-compare]
ConfigFile::Operation::READ == ConfigFile::State::READ
~~~~~~~~~~~~~~~~~~~~~~~~~~~ ^  ~~~~~~~~~~~~~~~~~~~~~~~
input_line_24:3:29: error: invalid operands to binary expression ('__cling_N514::ConfigFile::Operation' and '__cling_N514::ConfigFile::State')
ConfigFile::Operation::READ == ConfigFile::State::READ
~~~~~~~~~~~~~~~~~~~~~~~~~~~ ^  ~~~~~~~~~~~~~~~~~~~~~~~


Interpreter Error: 

An additional benefit comes from forcing external references to specify the full "path" to any enum value, and so the enum name itself is always there to provide context for the value. This is not the case for unscoped enums unless each enum value is named in a way that the context is unmistakeable. 

In [12]:
class Cat {
    public:
    enum EyeColour{GREEN, GREY, BLUE, BROWN};
};

// What did BROWN belong to? Is it the cat's fur colour, eye colour or something else?
int x = Cat::BROWN;

## Specifying cast expressions (pre-11)
C++ brings a dedicated syntax for casting variables between types via the operators:
* `static_cast<new_type>(variable)` for compile-time conversions (where a conversion function is available)
* `dynamic<new_type>(variable)` for runtime conversions where the type of the object is not necessarily known
  * _often used for class inheritance hierarchies, returns null if no conversion is possible_
* `const_cast<new_type>(variable)` for modifying the const-ness of the variable
* `reinterpret_cast<new_type>(variable)` for simply treating the underlying bit representation as a different type

But due to its backwards-compatibility with C, the language allows you to attempt legacy C-style casting:
* `(new_type)variable`, or commonly `(new_type*)variable_ptr` for inheritance hierarchies

Many who have started with C and moved into C++ continue to use C-style casting and perhaps might not know about the C++ cast expressions, but the behaviour of this is actually a little complicated behind the scenes.

[As described in cppreference.com](https://en.cppreference.com/w/cpp/language/explicit_cast), "when the C-style cast expression is encountered, the compiler attempts to interpret it as the following cast expressions, in this order":
1. const_cast<new_type>(expression);
2. static_cast<new_type>(expression), with extensions: pointer or reference to a derived class is additionally allowed to be cast to pointer or reference to unambiguous base class (and vice versa) even if the base class is inaccessible (that is, this cast ignores the private inheritance specifier). Same applies to casting pointer to member to pointer to member of unambiguous non-virtual base;
3. static_cast (with extensions) followed by const_cast;
4. reinterpret_cast<new_type>(expression);
5. reinterpret_cast followed by const_cast.

Chances are this isn't what we want. Rather than deciding consciously to work through a list of attempts, we probably have a particular type of cast that we are intending to do. So for improved readability and clarity of our intent we should simply specify that particular cast expression. With the various potential behaviours that the C-style cast can end up becoming, the consistent use of this cast throughout our codebase results in potentially many similarly-appearing expressions that perform different behaviour under the hood.